In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# !pip install -U tensorflow==2.0.0
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

TensorFlow version: 2.12.0
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
def parse (sent):
    words = word_tokenize(sent)
    parsed_words = [word for word in words if not word in stopwords.words()]
    return " ".join(parsed_words)

In [3]:
import csv
from tqdm import tqdm
labels = []
sentences = []
test_id = []
test_sentences = []
with open('/kaggle/input/nlp-getting-started/train.csv', 'r') as file:
    csv_reader = csv.reader(file)
    print(next(csv_reader))
    for row in tqdm(csv_reader):
        labels.append(int(row[4]))
#         sentences.append(row[3])
        sentences.append(parse(row[3]))
with open('/kaggle/input/nlp-getting-started/test.csv', 'r') as test_file:
    test_csv_reader = csv.reader(test_file)
    print(next(test_csv_reader))
    for tst_row in tqdm(test_csv_reader):
        test_id.append(tst_row[0])
#         test_sentences.append(tst_row[3])
        test_sentences.append(parse(tst_row[3]))

['id', 'keyword', 'location', 'text', 'target']


7613it [15:09,  8.37it/s]


['id', 'keyword', 'location', 'text']


3263it [06:34,  8.27it/s]


In [4]:
print(len(labels), len(sentences), len(test_sentences))

7613 7613 3263


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
tokenizer  = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
pad_seq = pad_sequences(sequences, maxlen = 30, padding = 'post', truncating  = 'post')
print(pad_seq.shape)
print(pad_seq[0])
i = np.array(pad_seq)
o = np.array(labels)
# print(tokenizer.word_index)
# print(tokenizer.index_word)
test_i = np.array(pad_sequences(tokenizer.texts_to_sequences(test_sentences), maxlen = 30, padding = 'post', truncating = 'post'))

(7613, 30)
[ 551 4419  717  167  834 1438 4420    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(50000, 10, input_length = 30),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation  = 'sigmoid')
])
model.summary()
model.compile( loss = 'binary_crossentropy', metrics = 'acc', optimizer = 'rmsprop')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 10)            500000    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               11008     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 513,121
Trainable params: 513,121
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(x= i, y= o, epochs = 10 ,validation_split=0.3)
print(history.params)

Epoch 1/10
167/167 [==============================] - 9s 27ms/step - loss: 0.6490 - acc: 0.6269 - val_loss: 0.6326 - val_acc: 0.6331
Epoch 2/10
167/167 [==============================] - 4s 24ms/step - loss: 0.4750 - acc: 0.7921 - val_loss: 0.5133 - val_acc: 0.7574
Epoch 3/10
167/167 [==============================] - 4s 21ms/step - loss: 0.3475 - acc: 0.8611 - val_loss: 0.5545 - val_acc: 0.7137
Epoch 4/10
167/167 [==============================] - 4s 21ms/step - loss: 0.2964 - acc: 0.8842 - val_loss: 0.4978 - val_acc: 0.7802
Epoch 5/10
167/167 [==============================] - 3s 21ms/step - loss: 0.2561 - acc: 0.9017 - val_loss: 0.5018 - val_acc: 0.7767
Epoch 6/10
167/167 [==============================] - 4s 21ms/step - loss: 0.2250 - acc: 0.9186 - val_loss: 0.5646 - val_acc: 0.7618
Epoch 7/10
167/167 [==============================] - 4s 22ms/step - loss: 0.1985 - acc: 0.9287 - val_loss: 0.6415 - val_acc: 0.6940
Epoch 8/10
167/167 [==============================] - 4s 21ms/step - 

In [8]:
pred_o = model.predict(test_i)
e = []
i=0
for oo in pred_o:
    if oo>0.5 :
        e.append([test_id[i],1])
    else:
        e.append([test_id[i],0])
    i = i+1
with open('/kaggle/working/output.csv', 'w') as op:
    write = csv.writer(op)
    write.writerow(["id", "target"])
    write.writerows(e)
# with open('/kaggle/working/output.csv', 'r') as opp:
#     re = csv.reader(opp)
#     for row in re:
#         print(row)

102/102 [==============================] - 1s 6ms/step
